# From Neurons to Networks: Exploring Deep Learning Algorithms

Roderick Perez, Ph.D.
OMV

---

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

1. Download the Solar Sunspot Dataset

In [2]:
# Download dataset (monthly sunspot data)
filePath = 'sunspot_data.csv'
data = pd.read_csv(filePath)

In [3]:
# Display the first few rows
data.head()

,Unnamed: 0,Year,Month,Day,Date In Fraction Of Year,Number of Sunspots,Standard Deviation,Observations,Indicator
0,0,1818,1,1,1818.001,-1,-1.0,0,1
1,1,1818,1,2,1818.004,-1,-1.0,0,1
2,2,1818,1,3,1818.007,-1,-1.0,0,1
3,3,1818,1,4,1818.010,-1,-1.0,0,1
4,4,1818,1,5,1818.012,-1,-1.0,0,1


In [4]:
# Filter and preprocess the dataset to focus on "Number of Sunspots"
data = data[data['Number of Sunspots'] >= 0]  # Remove negative values or missing entries
sunspots = data['Number of Sunspots'].values.reshape(-1, 1)

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
sunspots_normalized = scaler.fit_transform(sunspots)

NameError: name 'MinMaxScaler' is not defined

In [ ]:
print(f"Filtered data shape: {data.shape}")
print(f"Sunspots shape: {sunspots.shape}")

2. Plot the Dataset Using Matplotlib

In [ ]:
# Plot the dataset
plt.figure(figsize=(15, 5))
plt.plot(filtered_data['Date In Fraction Of Year'], sunspots, label='Sunspots', linewidth = 0.1)
plt.title('Filtered Solar Sunspot Dataset')
plt.xlabel('Time (Year)')
plt.ylabel('Number of Sunspots')
plt.legend()
plt.grid(True)
plt.show()

---

In [ ]:
!pip install torch

In [24]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [25]:
# Create windowed dataset
def create_windowed_dataset(data, window_size):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

In [26]:
window_size = 12  # You can experiment with this value
X, y = create_windowed_dataset(sunspots_normalized, window_size)

In [27]:
# Convert the data to PyTorch tensors
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)

In [28]:
# Modify the LSTM class to include dropout for regularization
class SunspotLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=50, output_size=1):
        super(SunspotLSTM, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.dropout = nn.Dropout(p=0.2)  # Adding dropout with 20% probability
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        lstm_out = self.dropout(lstm_out)  # Apply dropout to the LSTM output
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [29]:
# Hyperparameters
learning_rate = 0.0001 # Before was 0.001
weight_decay = 1e-5  # NEW parameters. Apply weight decay (L2 regularization)
epochs = 100  # You can increase this value for better training
input_size = 1
hidden_layer_size = 50
output_size = 1

#################### NEW ####################
# Early stopping criteria
early_stopping_patience = 10  # Stop training if no improvement for 10 epochs
best_loss = float('inf')
patience_counter = 0

In [30]:
# Initialize the model, loss function, and optimizer
model = SunspotLSTM(input_size=input_size, hidden_layer_size=hidden_layer_size, output_size=output_size)
loss_function = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Before
model = SunspotLSTM(input_size=input_size, hidden_layer_size=hidden_layer_size, output_size=output_size)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)  # Apply weight decay

In [ ]:
# Training loop with gradient clipping
losses = []
for epoch in range(epochs):
    for seq, labels in zip(X, y):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()

        # Apply gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

    losses.append(single_loss.item())

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {single_loss.item()}')

    ###### NEW ############ Early stopping
    if single_loss.item() < best_loss:
        best_loss = single_loss.item()
        patience_counter = 0  # Reset counter if improvement
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print(f"Stopping early at epoch {epoch} due to no improvement in loss.")
            break

In [ ]:
# Plot the training loss
plt.figure(figsize=(10, 5))
plt.plot(losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [33]:
# Generating predictions for the dataset
model.eval()
predictions = []
for seq in X:
    with torch.no_grad():
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))
        predictions.append(model(seq).item())

In [34]:
# Inverse transform the normalized values back to original scale
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

In [ ]:
# Plot original data vs predictions
plt.figure(figsize=(10, 5))
plt.plot(data['Year'], sunspots, label='Original Sunspot Data')
plt.plot(data['Year'][window_size:], predictions, label='Predicted Sunspots', linestyle='dashed')
plt.xlabel('Year')
plt.ylabel('Number of Sunspots')
plt.title('Original vs Predicted Sunspots')
plt.legend()
plt.grid(True)
plt.show()

Save the model

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'sunspot_lstm_model_V3.pth')

Load the model

In [ ]:
# Recreate the model architecture
model = SunspotLSTM(input_size=input_size, hidden_layer_size=hidden_layer_size, output_size=output_size)

# Load the saved model parameters
model.load_state_dict(torch.load('sunspot_lstm_model_V3.pth'))

# Set the model to evaluation mode (important for inference)
model.eval()

Use the Loaded Model for Prediction

In [ ]:
# Generate predictions with the loaded model
predictions = []
for seq in X:
    with torch.no_grad():
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))
        predictions.append(model(seq).item())